In [6]:
import pandas as pd

data = pd.read_excel("solar_data.xlsx", header=3).T
data

,0,1,2,3,4,5,6,7,8,9,...,2789,2790,2791,2792,2793,2794,2795,2796,2797,2798
Small Unit Installation Postcode,0.00,200.00,800.000,801.000,803.000,804.00,810.000,811.0,812.000,813.000,...,7330.000,7331.00,7466.00,7467.00,7468.000,7469.000,7470.000,7802.00,8576.00,9729.000
Previous Years (2001-2018) - Installations Quantity,4.00,1.00,55.000,7.000,2.000,14.00,1991.000,2.0,1020.000,6.000,...,303.000,23.00,4.00,64.00,28.000,34.000,28.000,1.00,2.00,1.000
Previous Years (2001-2018) - SGU Rated Output In kW,4.46,0.08,1338.570,134.868,11.685,68.95,11293.086,9.0,5630.618,78.949,...,1771.722,120.95,10.38,309.79,170.615,119.395,103.872,2.45,6.84,0.875
Jan 2019 - Installations Quantity,0.00,0.00,0.000,0.000,0.000,0.00,31.000,0.0,26.000,0.000,...,1.000,0.00,0.00,2.00,0.000,0.000,1.000,0.00,0.00,0.000
Jan 2019 - SGU Rated Output In kW,0.00,0.00,0.000,0.000,0.000,0.00,207.720,0.0,163.025,0.000,...,5.490,0.00,0.00,10.59,0.000,0.000,5.500,0.00,0.00,0.000
Feb 2019 - Installations Quantity,0.00,0.00,2.000,0.000,0.000,0.00,41.000,0.0,33.000,0.000,...,3.000,0.00,0.00,0.00,2.000,0.000,1.000,0.00,0.00,0.000
Feb 2019 - SGU Rated Output In kW,0.00,0.00,13.090,0.000,0.000,0.00,263.065,0.0,252.610,0.000,...,25.340,0.00,0.00,0.00,6.900,0.000,6.490,0.00,0.00,0.000
Mar 2019 - Installations Quantity,0.00,0.00,3.000,0.000,0.000,0.00,77.000,0.0,31.000,0.000,...,1.000,0.00,0.00,0.00,0.000,0.000,1.000,0.00,0.00,0.000
Mar 2019 - SGU Rated Output In kW,0.00,0.00,19.800,0.000,0.000,0.00,514.455,0.0,223.800,0.000,...,5.100,0.00,0.00,0.00,0.000,0.000,6.600,0.00,0.00,0.000
Apr 2019 - Installations Quantity,0.00,0.00,1.000,0.000,0.000,0.00,50.000,0.0,25.000,0.000,...,1.000,0.00,0.00,0.00,0.000,0.000,1.000,0.00,0.00,0.000
